In [22]:
from common import *
from process_ntuples import *
from process_df import *
from cuts import apply_cuts, apply_pt_cut

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras import layers

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from tensorflow.keras.callbacks import EarlyStopping

#weights and biases imports
import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint, WandbCallback

import random
import numpy as np
import tensorflow as tf

In [5]:
dir_path = "/global/cfs/cdirs/atlas/gottino/Charm/Analysis/run_batch_gup/ntuples"

files = os.listdir(dir_path)
TTree = process_ntuple(dir_path, files[0])
files

['CharmAnalysis.mc.ForcedDecay.Dplus.tree.root',
 'CharmAnalysis.mc.ForcedDecay.LabmdaC.tree.root']

$D^{+}$ : 411

$\Lambda_C^{+}$ : 4122

Apply Cuts Fiducial Region cuts ($𝐷^{+}$ $𝑝_T$ > 8 GeV, |𝜂| < 2.2). ***See cuts.py on how the cuts are applied***

In [ ]:
DMesons_df = show(0, TTree, ["DMesons_pt", "DMesons_fitOutput__Lxy", "DMesons_costhetastar", "DMesons_pdgId"], 20000000)
DMesons_df = remove_empty(DMesons_df)

DMesons_df["DMesons_pt"] = DMesons_df["DMesons_pt"].apply(divide_by_1000)
for col in DMesons_df.columns:
    DMesons_df[col] = DMesons_df[col].apply(list)
    
DMesons_df = apply_pt_cut(DMesons_df)
pd.set_option('display.max_rows', 10)
DMesons_df

,DMesons_pt,DMesons_fitOutput__Lxy,DMesons_costhetastar,DMesons_pdgId
0,[10.047125],[2.3395302295684814],[0.25487038493156433],[411]
1,"[18.9937421875, 18.9937421875]","[1.9184143543243408, 1.9184143543243408]","[-0.36730244755744934, -0.0650731548666954]","[411, 4122]"
2,"[10.118494140625, 10.118494140625]","[2.8901257514953613, 2.8901257514953613]","[0.06833142042160034, 0.4276972711086273]","[411, 4122]"
3,"[12.579470703125, 12.579470703125]","[0.9249186515808105, 0.9249186515808105]","[-0.5603189468383789, -0.41332852840423584]","[411, 4122]"
4,[36.1288359375],[0.3584118187427521],[0.19036155939102173],[4122]
...,...,...,...,...
8234903,[17.96],[5.87862491607666],[0.45278045535087585],[411]
8234904,"[62.91061328125, 62.91061328125]","[11.870012283325195, 11.870012283325195]","[-0.9544855356216431, -0.9355815052986145]","[411, 4122]"
8234905,"[14.8181767578125, 14.8181767578125]","[2.027026653289795, 2.027026653289795]","[-0.9712575674057007, -0.9517184495925903]","[411, 4122]"
8234906,[39.3814296875],[0.17065481841564178],[-0.332907497882843],[4122]


In [7]:
DMesons_df2 = show(0, TTree, ["DMesons_daughterInfo__pt", "DMesons_daughterInfo__eta", "DMesons_daughterInfo__phi", "DMesons_daughterInfo__pdgId"], 100)
DMesons_df2 = remove_empty(DMesons_df2)

In [9]:
DPlus = [-321, 211, 211]
LambdaC = [2212, -321, 211]

remove_lambda = []

count = 0
for index, row in DMesons_df2.iterrows():
    if count == 50:
        break
    count+=1
    
    pts = row['DMesons_daughterInfo__pt']
    etas = row['DMesons_daughterInfo__eta']
    phis = row['DMesons_daughterInfo__phi']
    pdgids = row["DMesons_daughterInfo__pdgId"]
    
    #loop through until the first lambda encountered
    for i in range(len(pdgids) - 1):
        #if there are multiple lambda baryons
        if len(pdgids) > 2:
            #stop at the first lambda and keep the higher proton pt valued lambda
            if list(pdgids[i]) == LambdaC:
                l1_pt, l2_pt = pts[i][0], pts[i + 1][0]
                if (l1_pt > l2_pt):
                    #Lambda 1's proton has higher pt
                    remove_lambda.append((index, i + 1))
                elif (l1_pt < l2_pt):
                    #Lambda 2's proton has higher pt!
                    remove_lambda.append((index, i))
                else:
                    raise Exception("Error: Are the proton pt vaules equal?")
                    
len(remove_lambda)

3

In [24]:
#takes tuples of shape (2,) / (index, column)
def remove_particles_from_df(df, tuple_list):
    for tup in tuple_list:
        index, column = tup
        print(index, column)
        
        new_df = df.copy()
        for i in range(len(DMesons_df.iloc[index, 0][column]) + 1):
            #new_df["DMesons_daughterInfo__pt"] = new_df["DMesons_daughterInfo__pt"].apply(lambda x: print(x))
            val = new_df.iloc[index, i][column]
            print(val)
    df = df.drop(index, axis = 0)
    return df
#df = remove_particles_from_df(DMesons_df, remove_lambda)

#for now just remove the rows
indices = [i for i, x in enumerate(DMesons_df2[DMesons_df2.columns[0]]) if len(x) > 2]
DMesons_df2 = DMesons_df2.drop(indices)
DMesons_df2 = DMesons_df2.reset_index(drop=True)
DMesons_df2["DMesons_daughterInfo__pt"] = DMesons_df2["DMesons_daughterInfo__pt"].apply(divide_by_1000)
DMesons_df2

pd.set_option('display.max_rows', 10)
# pd.set_option('display.max_columns', None)

# print("-----------------")
# print(DMesons_df.iloc[32, 0][2])
# print(DMesons_df.iloc[32, 1][2])
# print(DMesons_df.iloc[32, 2][2])
# print(DMesons_df.iloc[32, 3][2])

In [10]:
# create a new column 'particle_type' based on the pdgId column
func = lambda x: 'only_411' if x == [411] else ('only_4122' if x == [4122] else 'both')
DMesons_df['particle_type'] = DMesons_df['DMesons_pdgId'].apply(func)

# separate the rows based on the new column
# only_411 = DMesons_df[DMesons_df['particle_type'] == 'only_411']
# only_4122 = DMesons_df[DMesons_df['particle_type'] == 'only_4122']
# both = DMesons_df[DMesons_df['particle_type'] == 'both']

new_df = DMesons_df[DMesons_df['particle_type'] != 'both']
new_df = new_df.reset_index(drop=True)
new_df.loc[:, 'DMesons_pt'] = new_df['DMesons_pt'].apply(lambda x: float(x[0]))
new_df.loc[:, 'DMesons_fitOutput__Lxy'] = new_df['DMesons_fitOutput__Lxy'].apply(lambda x: float(x[0]))
new_df.loc[:, 'DMesons_costhetastar'] = new_df['DMesons_costhetastar'].apply(lambda x: float(x[0]))
new_df.loc[:, 'DMesons_pdgId'] = new_df['DMesons_pdgId'].apply(lambda x: int(x[0]))
new_df = new_df.drop('particle_type', axis=1)
new_df

/tmp/ipykernel_4188/230995059.py:12: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  new_df.loc[:, 'DMesons_pt'] = new_df['DMesons_pt'].apply(lambda x: float(x[0]))
/tmp/ipykernel_4188/230995059.py:13: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  new_df.loc[:, 'DMesons_fitOutput__Lxy'] = new_df['DMesons_fitOutput__Lxy'].apply(lambda x: float(x[0]))
/tmp/ipykernel_4188/230995059.py:14: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the 

,DMesons_pt,DMesons_fitOutput__Lxy,DMesons_costhetastar,DMesons_pdgId
0,10.047125,2.339530,0.254870,411
1,36.128836,0.358412,0.190362,4122
2,14.835231,2.899070,0.684572,411
3,8.800141,0.246351,-0.737246,4122
4,30.092365,7.426617,-0.937342,4122
...,...,...,...,...
2516358,24.477744,3.593372,0.539289,411
2516359,14.971446,2.374407,0.863583,411
2516360,12.734207,5.151422,0.461929,411
2516361,17.960000,5.878625,0.452780,411


In [11]:
D = 0
Lambda = 0
for pdgId in new_df["DMesons_pdgId"]:
    if pdgId == 411:
        D+=1
    elif pdgId == 4122:
        Lambda+=1
print(D, Lambda)

1566841 949522


In [13]:
df_norm = (new_df - new_df.min()) / (new_df.max() - new_df.min())
df_norm.loc[:, 'DMesons_pdgId'] = df_norm['DMesons_pdgId'].apply(lambda x: int(x))
df_norm

X_train, X_test, y_train, y_test = train_test_split(new_df[['DMesons_pt', 'DMesons_fitOutput__Lxy', 'DMesons_costhetastar']], df_norm['DMesons_pdgId'], test_size=0.2, random_state=42)

/tmp/ipykernel_4188/3547295190.py:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_norm.loc[:, 'DMesons_pdgId'] = df_norm['DMesons_pdgId'].apply(lambda x: int(x))


In [14]:
X_train

,DMesons_pt,DMesons_fitOutput__Lxy,DMesons_costhetastar
1393567,8.998823,0.415244,-0.676589
247341,16.169950,0.432640,0.822002
1720792,23.583254,1.711358,0.509549
235015,31.431924,1.232870,0.380954
1760004,24.270182,3.152954,0.582256
...,...,...,...
110268,10.439533,2.331200,0.816670
1692743,11.541045,3.062604,0.980448
2356330,8.304547,0.295519,0.010769
2229084,23.810479,4.198989,0.724207


In [23]:
# only_411 = DMesons_df[DMesons_df['DMesons_pdgId'].apply(lambda x: x == [411])]
# only_4122 = DMesons_df[DMesons_df['DMesons_pdgId'].apply(lambda x: x == [4122])]
# both = DMesons_df[DMesons_df['DMesons_pdgId'].apply(lambda x: len(x) == 2)]

In [19]:
wandb.init(
    # set the wandb project where this run will be logged
    project="ATLAS_DNN",

    # track hyperparameters and run metadata with wandb.config
    config={
        "layer_1": 64,
        "activation_1": "relu",
        "dropout": 0.5,
        "layer_2": 32,
        "activation_2": "relu",
        "dropout": 0.5,
        "output_layer": 1,
        "output_activation": "sigmoid",
        "optimizer": "adam",
        "loss": "binary_crossentropy",
        "metric": "accuracy",
        "epoch": 50,
        "batch_size": 32
    }
)

config = wandb.config

In [20]:
#model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [23]:
#Compile
model.compile(optimizer='adam', loss='binary_crossentropy')

early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='min')
# checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True, verbose=1)
# tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=False)
#Train
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stop, WandbCallback(), tf.keras.callbacks.History()])

#Evaluate testing set
loss = model.evaluate(X_test, y_test)
print("Loss: {}".format(loss))

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/50
62882/62910 [============================>.] - ETA: 0s - loss: 0.1949

2023-04-03 02:40:11.930598: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,64]
	 [[{{node inputs}}]]
2023-04-03 02:40:11.946263: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,32]
	 [[{{node inputs}}]]
2023-04-03 02:40:12.116731: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,64]
	 [[{{node inputs}}]]
2023-04-03 02:40:12

INFO:tensorflow:Assets written to: /global/u2/g/gupsingh/ATLAS_DNN/wandb/run-20230403_023046-gvsm5bl2/files/model-best/assets


INFO:tensorflow:Assets written to: /global/u2/g/gupsingh/ATLAS_DNN/wandb/run-20230403_023046-gvsm5bl2/files/model-best/assets
wandb: Adding directory to artifact (/global/u2/g/gupsingh/ATLAS_DNN/wandb/run-20230403_023046-gvsm5bl2/files/model-best)... Done. 0.1s


62910/62910 [==============================] - 103s 2ms/step - loss: 0.1949 - val_loss: 0.1627
Epoch 2/50
62910/62910 [==============================] - 101s 2ms/step - loss: 0.1716 - val_loss: 0.1638
Epoch 3/50
62910/62910 [==============================] - 101s 2ms/step - loss: 0.1694 - val_loss: 0.1636
Epoch 4/50
62899/62910 [============================>.] - ETA: 0s - loss: 0.1680

2023-04-03 02:45:16.501415: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,64]
	 [[{{node inputs}}]]
2023-04-03 02:45:16.514422: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,32]
	 [[{{node inputs}}]]
2023-04-03 02:45:16.646987: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,64]
	 [[{{node inputs}}]]
2023-04-03 02:45:16

INFO:tensorflow:Assets written to: /global/u2/g/gupsingh/ATLAS_DNN/wandb/run-20230403_023046-gvsm5bl2/files/model-best/assets


INFO:tensorflow:Assets written to: /global/u2/g/gupsingh/ATLAS_DNN/wandb/run-20230403_023046-gvsm5bl2/files/model-best/assets
wandb: Adding directory to artifact (/global/u2/g/gupsingh/ATLAS_DNN/wandb/run-20230403_023046-gvsm5bl2/files/model-best)... Done. 0.0s


62910/62910 [==============================] - 102s 2ms/step - loss: 0.1680 - val_loss: 0.1565
Epoch 5/50
62910/62910 [==============================] - 101s 2ms/step - loss: 0.1668 - val_loss: 0.1594
Epoch 6/50
62910/62910 [==============================] - 101s 2ms/step - loss: 0.1667 - val_loss: 0.1621
Epoch 7/50
62910/62910 [==============================] - 104s 2ms/step - loss: 0.1660 - val_loss: 0.1601
Epoch 7: early stopping
15728/15728 [==============================] - 17s 1ms/step - loss: 0.1601
Loss: 0.16012655198574066


In [24]:
wandb.finish()
predictions = model.predict(X_test)
predictions

epoch,▁▂▃▅▆▇█
loss,█▂▂▁▁▁▁
val_loss,▇██▁▄▆▄
best_epoch,3
best_val_loss,0.1565
epoch,6
loss,0.16596
val_loss,0.16013


15728/15728 [==============================] - 15s 952us/step


array([[0.04383187],
       [1.        ],
       [0.04932335],
       ...,
       [1.        ],
       [0.04383187],
       [0.04383187]], dtype=float32)

In [1]:
predictions_list = []
D_count = 0
Lambda_count = 0
unclear = 0
for pred in predictions:
    if float(pred) > 0.9:
        predictions_list.append(int(1))
        Lambda_count+=1
    elif float(pred) < 0.9:
        predictions_list.append(int(0))
        D_count+=1
    else:
        unclear+=1
        
print("$D^+$ Count: {}".format(D_count))
print("$\Lambda_C^+ Count: {}".format(Lambda_count))
print("Unknown Particle Count: {}".format(unclear))
np.shape(predictions)

NameError: name 'predictions' is not defined

In [32]:
print(predictions[:10])

[[0.04383187]
 [1.        ]
 [0.04932335]
 [0.04383187]
 [1.        ]
 [0.04383187]
 [0.04383187]
 [0.0538348 ]
 [0.04383187]
 [0.04383187]]


In [ ]:
accuracy = np.mean(np.round(predictions) == y_test)
print("Accuracy: {:.2%}".format(accuracy))